In [5]:
%run Archpool.ipynb
%run Topo_treatment.ipynb

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image

import model
from dataset import *
from utils import check_dir
from tqdm import tqdm
import numpy as np
from functools import reduce

import os
import json
import argparse

import sys
import json
sys.path.insert(0, './persis_lib_cpp')
from persis_homo_optimal import *

torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def parse_args():
    
    '''f = open('example.json')
    config = json.load(f)'''
    config = return_config()

    try:
        if config['output_path'][-1] != '/':
            config['output_path'] += '/'
        if config['train_data_path'][-1] != '/':
            config['train_data_path'] += '/'
        if config['val_data_path'][-1] != '/':
            config['val_data_path'] += '/'
    except KeyError as err:
        print(f'{opt.config}: Unspecified path {err}')
        exit(1)
    return config


def initialize_network(config):
    network = {}
    random_seed = 0
    try:
        config['resume']
        random_seed = config['random_seed']
    except KeyError:
        config['resume'] = False
    torch.manual_seed(random_seed)
    if (config['resume'] and os.path.isfile(config['resume'])):
        confout = config['output_path'] + config['name'] + '/'
        network['resume'] = True
        checkpoint = torch.load(config['resume'], map_location=torch_device)
        network['epoch_start'] = checkpoint['epoch'] + \
            1 if checkpoint['output_dir'] == confout else 0
        network['epoch_end'] = config['epoch'] or checkpoint['epoch_end']
        network['output_dir'] = confout
        network['checkpoint_dir'] = checkpoint['checkpoint_dir']
        network['learning_rate'] = checkpoint['learning_rate']
        network['train_data_dir'] = checkpoint['train_data_dir']
        network['val_data_dir'] = checkpoint['val_data_dir']
        network['name'] = checkpoint['name']
        network['batch_size'] = checkpoint['batch_size']
        network['features'] = checkpoint['features']
        network['image_size'] = checkpoint['image_size']
        network['image_channels'] = checkpoint['image_channels']
        network['optimizer_name'] = checkpoint['optimizer_name']
        network['arch'] = checkpoint['arch']
        network['bn'] = checkpoint['bn']
        network['checkpoint'] = checkpoint
    else:
        network['resume'] = False
        try:
            network['output_dir'] = config['output_path'] + \
                config['name'] + '/'
            network['name'] = config['name']
            network['epoch_start'] = 0
            network['epoch_end'] = config['epoch']
            network['learning_rate'] = config['learning_rate']
            network['batch_size'] = config['batch_size']
            network['features'] = int(config['features'])
            network['image_size'] = config['image_size']
            network['image_channels'] = config['image_channels']
            network['optimizer_name'] = config['optimizer']
            network['train_data_dir'] = config['train_data_path']
            network['val_data_dir'] = config['val_data_path']
            network['arch'] = config['arch']
            network['bn'] = config['bn']
        except KeyError as err:
            print(f'Configuration: Unspecified field {err}')
            exit(1)
    network['checkpoint_dir'] = network['output_dir'] + 'checkpoints/'
    network['result_dir'] = network['output_dir'] + 'result/'
    check_dir(network['output_dir'])
    check_dir(network['checkpoint_dir'])
    check_dir(network['result_dir'])

    network['logfile_path'] = network['result_dir'] + 'logfile.txt'
    network['performance_path'] = network['result_dir'] + 'performance.txt'
    learning_model = model.AutoEncoder(
        network['image_size'], network['image_channels'], network['features'], network['arch'], network['bn'])
    learning_model = learning_model.to(torch_device)
    
    network['loss_function'] = WeightedBCELoss(one_weight=1,zeros_weight=1)
    # network['loss_function'] = nn.MSELoss()
    optimizer = None
    if network['optimizer_name'] == 'adam':
        optimizer = optim.Adam(learning_model.parameters(),
                               lr=network['learning_rate'])
    elif network['optimizer_name'] == 'sgd':
        optimizer = optim.SGD(learning_model.parameters(),
                              momentum=0.9, weight_decay=1e-2,
                              lr=network['learning_rate'])

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=1)

    if (network['resume']):
        learning_model.load_state_dict(network['checkpoint']['model'])
        optimizer.load_state_dict(network['checkpoint']['optimizer'])
        scheduler.load_state_dict(network['checkpoint']['scheduler'])
    
    network['model'] = learning_model
    network['optimizer'] = optimizer
    network['scheduler'] = scheduler
    return network


class WeightedBCELoss:
    def __init__(self, one_weight=1.0, zeros_weight=1.0, reduction="mean"):
        self.reduction = reduction
        self.update_weights(one_weight, zeros_weight)

    def update_weights(self, one_weight, zeros_weight):
        self.weights = torch.FloatTensor([one_weight, zeros_weight])
        self.weights.to(torch_device)

    def _bce(self, x, y):
        weights = -self.weights
        x = torch.clamp(x, min=1e-7, max=1-1e-7)
        y = torch.clamp(y, min=1e-7, max=1-1e-7)
        return weights[1]*y*torch.log(x) + weights[0]*(1-y)*torch.log(1-x)

    def __call__(self, pred, truth):
        loss = self._bce(pred, truth)
        if self.reduction == 'mean':
            return torch.mean(loss)
        if self.reduction == 'sum':
            return torch.sum(loss)
        return loss



def train(adv_params, network, dataloader, withTopo=False):
    
    et = Edges_(adv_params, debug=False)
    #criterionT = GANLoss("vanilla_topo", "sum").to(torch_device)
    criterionT = nn.BCELoss(reduction="sum")
    loss_function = network['loss_function']
    model = network['model']
    optimizer = network['optimizer']
    model.train()
    running_loss = 0.0
    t_loss = 0.0
    result_dir = network['result_dir']
    batch_number = 0
    step_num = 0
    tot_append_loss = 0.0
    for data in dataloader:
        
        scalars, label = data

        scalars = scalars.to(torch_device)
        label = label.to(torch_device)
        optimizer.zero_grad()
        
        p = model(scalars)
        #prediction = ((p - torch.min(p))/(torch.max(p) - torch.min(p)))*2-1
        
        loss = loss_function(p, label)
        running_loss += loss.item()
        
        num_rows = p.size(0)
        
        batch_number += 1
        print("batch_number : ", batch_number)
        
        if withTopo:
            
            tp_wgt   = et.return_tp_weight()
            fake_fix, mean_wasdis = et.fix_with_topo(p.detach().cpu().numpy(), label.detach().cpu().numpy(),
                                                     et.return_target_dim(), result_dir, batch_number, num_rows, 
                                                     0.0, 1.0, blind=et.blind())
            
            fake_fix = torch.from_numpy(fake_fix).to(torch_device)
            
            fake_fix = torch.unsqueeze(fake_fix, 1)
            fake_fix = (fake_fix - torch.min(fake_fix))/(torch.max(fake_fix) - torch.min(fake_fix))
            errT = criterionT(p, fake_fix) * tp_wgt
            t_loss += errT.item()
            tot_loss = loss + errT
            tot_append_loss += tot_loss.item()

            tot_loss.backward()
            optimizer.step()
        
        else:
            loss.backward()
            optimizer.step()
    
    if withTopo:
        training_loss = running_loss / len(dataloader.dataset)
        topo_loss = t_loss / len(dataloader.dataset)
        total_loss = tot_append_loss / len(dataloader.dataset)
    
        return [training_loss], [topo_loss], [total_loss]
    else:
        training_loss = running_loss / len(dataloader.dataset)
        return [training_loss]


def validate(network, dataloader, epoch):
    image_size = [network['image_size'], network['image_size']]
    running_loss = 0.0
    tp = 0.0  # true positive
    tn = 0.0  # true negative
    fp = 0.0  # false positive
    fn = 0.0  # false negative

    l1_diff = 0.0
    with torch.no_grad():
        loss_function = network['loss_function']
        model = network['model']
        result_dir = network['result_dir']
        image_channels = network['image_channels']
        model.eval()
        batch_number = 0
        output_image = False
        save_output = False
        
        if epoch != "":
            if (epoch != 0) and (epoch != 10) and ((epoch+1)%10 == 0):        
                # create directory for the epoch
                # add lines
                epoch_dir = os.path.join(network['output_dir'], f"epoch_{epoch}")
                if not os.path.exists(epoch_dir):
                    os.makedirs(epoch_dir)
                save_output = True
        
        for i, data in enumerate(dataloader):
            scalars, label = data
            label = label.to(torch_device)
            scalars = scalars.to(torch_device)
            batch_size = label.size(0)

            prediction = model(scalars)
            
            loss = loss_function(prediction, label)
            running_loss += loss.item() * batch_size
            # log accuracy
            pred = prediction.cpu().view(batch_size, -1).double()
            truth = label.cpu().view(batch_size, -1).double()

            plabel = torch.zeros(pred.size())
            plabel[pred >= 0.3] = 1
            
            tp += torch.sum(torch.logical_and(plabel == 1, truth == 1).float())
            tn += torch.sum(torch.logical_and(plabel == 0, truth == 0).float())
            fp += torch.sum(torch.logical_and(plabel == 1, truth == 0).float())
            fn += torch.sum(torch.logical_and(plabel == 0, truth == 1).float())

            l1_diff += torch.sum(torch.abs(pred - truth))

            if epoch != "":
                if (epoch == 0) or (epoch == network['epoch_end'] - 1) or (i == len(dataloader) - 1) or (epoch == 3):
                    output_image = True

            if output_image:
                num_rows = batch_size
                s = scalars.cpu().view(
                    num_rows, 1, image_size[1], image_size[0]).double()
                t = label.cpu().view(
                    num_rows, 1, image_size[1], image_size[0]).double()

                pred = prediction.cpu().view(
                    num_rows, 1, image_size[1], image_size[0]).double()
                
                pl = plabel.cpu().view(
                    num_rows, 1, image_size[1], image_size[0]).double()
                
                out_image = torch.transpose(torch.stack((s, t, pl,pred)), 0, 1).reshape(
                    4*num_rows, 1,  image_size[1], image_size[0])
                save_image(out_image.cpu(
                ), f"{result_dir}epoch_{epoch}_batch{batch_number}.png", padding=4, nrow=24)
            batch_number += 1
            
            if save_output:
                # save the output as numpy array
                # add lines
                for j in range(len(prediction)):
                    p = np.squeeze(prediction[j].detach().cpu().numpy())
                    filename = os.path.join(epoch_dir, f"output_{i*len(prediction)+j:05}.npy")
                    np.save(filename, p)
                    
        # end for loop
    # end with nograd
    val_loss = running_loss/len(dataloader.dataset)
    l1_diff /= len(dataloader.dataset)
    tp /= len(dataloader.dataset)
    tn /= len(dataloader.dataset)
    fp /= len(dataloader.dataset)
    fn /= len(dataloader.dataset)
    accuracy = (tp + tn) / (tp + tn + fp + fn + 1e-9)
    precision = (tp)/(tp+fp + 1e-9)
    recall = (tp)/(tp+fn + 1e-9)
    f1 = 2*tp / (2 * tp + fp + fn + 1e-9)

    return [val_loss], [accuracy, precision, recall, f1, l1_diff]


def floats2str(l):
    return ",".join(map(lambda x: f'{x:.6f}', l))


def parameters_count(model):
    total = 0
    total_t = 0
    for p in model.parameters():
        if p.requires_grad:
            total += p.numel()
            total_t += p.numel()
        else:
            total += p.numel()
    return total, total_t


def main():
    adv_params  = return_advanced_params()
    config = parse_args()
    withTopo = False
    withTopo = config['withTopo']
    network = initialize_network(config)

    p, pt = parameters_count(network['model'])
    print(f'number of parameters(trainable) {p}({pt})')

    with open(network['output_dir']+'config.json', 'w') as jsonout:
        json.dump(config, jsonout, indent=2)

    train_dataset = ImageBoundary(
        config['train_data_path'], network['image_channels'])
    train_dataloader = DataLoader(
        train_dataset, batch_size=network['batch_size'], shuffle=True)
    val_dataset = ImageBoundary(
        config['val_data_path'], network['image_channels'])
    val_dataloader = DataLoader(
        val_dataset, batch_size=network['batch_size'], shuffle=False)

    '''if network['resume']:
        logfile = open(network['logfile_path'], 'a')
        perf_log = open(network['performance_path'], 'a')
    else:'''
    if withTopo:
        logfile = open(network['logfile_path'], 'w')
        logfile.write('epoch,train_loss,val_loss,topo_loss,total_loss\n')
    else:
        logfile = open(network['logfile_path'], 'w')
        logfile.write('epoch,train_loss,val_loss\n')
    
    perf_log = open(network['performance_path'], 'w')
    perf_log.write(
        'epoch, accuracy, precision, recall, f1, l1_diff_per_image\n')

    for epoch in tqdm(range(network['epoch_start'], network['epoch_end'])):
        if withTopo:
            t_loss, topo_loss, total_loss = train(adv_params, network, train_dataloader, True)
            v_loss, performance = validate(network, val_dataloader, epoch)
            network['scheduler'].step(total_loss[0])
            
            t_loss = floats2str(t_loss)
            v_loss = floats2str(v_loss)
            topo_loss = floats2str(topo_loss)
            total_loss = floats2str(total_loss)
            
            logfile.write(f'{epoch},{t_loss},{v_loss},{topo_loss},{total_loss}\n')
            logfile.flush()
        else:
            t_loss = train(adv_params, network, train_dataloader, False)
            v_loss, performance = validate(network, val_dataloader, epoch)
            network['scheduler'].step(t_loss[0])
            
            t_loss = floats2str(t_loss)
            v_loss = floats2str(v_loss)
            
            logfile.write(f'{epoch},{t_loss},{v_loss}\n')
            logfile.flush()
        
        performance = floats2str(performance)
        perf_log.write(f'{epoch},{performance}\n')
        perf_log.flush()
        
        if ((epoch+1) % 50 == 0) or epoch == network['epoch_end'] - 1:
            torch.save({
                'epoch': epoch,
                'epoch_end': network['epoch_end'],
                'model': network['model'].state_dict(),
                'optimizer': network['optimizer'].state_dict(),
                'optimizer_name': network['optimizer_name'],
                'scheduler': network['scheduler'].state_dict(),
                'checkpoint_dir': network['checkpoint_dir'],
                'train_data_dir': network['train_data_dir'],
                'val_data_dir': network['val_data_dir'],
                'output_dir': network['output_dir'],
                'name': network['name'],
                'batch_size': network['batch_size'],
                'learning_rate': network['learning_rate'],
                'features': network['features'],
                'image_size': network['image_size'],
                'image_channels': network['image_channels'],
                'arch': network['arch'],
                'bn': network['bn']
            }, f'{network["checkpoint_dir"]}{network["name"]}_{epoch}.pth')
    logfile.close()
    perf_log.close()

if __name__ == '__main__':
    main()

number of parameters(trainable) 7762465(7762465)


  0%|          | 0/100 [00:00<?, ?it/s]

batch_number :  1


c:\Users\syed_fahim_ahmed\anaconda3\envs\topo_loss\lib\site-packages\ipykernel_launcher.py:508: DeprecationWarning: Call to deprecated class PersImage. (Replaced with the class `persim.PersistenceImager`.) -- Deprecated since version 0.1.5.


batch_number :  2
batch_number :  3
batch_number :  4
batch_number :  5
batch_number :  6
batch_number :  7
batch_number :  8
batch_number :  9
batch_number :  10
batch_number :  11
batch_number :  12
batch_number :  13
batch_number :  14
batch_number :  15
batch_number :  16
batch_number :  17
batch_number :  18
batch_number :  19
batch_number :  20
batch_number :  21
batch_number :  22
batch_number :  23
batch_number :  24
batch_number :  25
batch_number :  26
batch_number :  27
batch_number :  28
batch_number :  29
batch_number :  30
batch_number :  31
batch_number :  32
batch_number :  33
batch_number :  34
batch_number :  35
batch_number :  36
batch_number :  37
batch_number :  38
batch_number :  39
batch_number :  40
batch_number :  41
batch_number :  42
batch_number :  43
batch_number :  44
batch_number :  45
batch_number :  46
batch_number :  47
batch_number :  48
batch_number :  49
batch_number :  50
batch_number :  51
batch_number :  52
batch_number :  53
batch_number :  54
b

...

batch_number :  1
batch_number :  2
batch_number :  3
batch_number :  4
batch_number :  5
batch_number :  6
batch_number :  7
batch_number :  8
batch_number :  9
batch_number :  10
batch_number :  11
batch_number :  12
batch_number :  13
batch_number :  14
batch_number :  15
batch_number :  16
batch_number :  17
batch_number :  18
batch_number :  19
batch_number :  20
batch_number :  21
batch_number :  22
batch_number :  23
batch_number :  24
batch_number :  25
batch_number :  26
batch_number :  27
batch_number :  28
batch_number :  29
batch_number :  30
batch_number :  31
batch_number :  32
batch_number :  33
batch_number :  34
batch_number :  35
batch_number :  36
batch_number :  37
batch_number :  38
batch_number :  39
batch_number :  40
batch_number :  41
batch_number :  42
batch_number :  43
batch_number :  44
batch_number :  45
batch_number :  46
batch_number :  47
batch_number :  48
batch_number :  49
batch_number :  50
batch_number :  51
batch_number :  52
batch_number :  53
ba

  3%|▎         | 3/100 [00:31<18:08, 11.23s/it]

batch_number :  1
batch_number :  2
batch_number :  3
batch_number :  4
batch_number :  5
batch_number :  6
batch_number :  7
batch_number :  8
batch_number :  9
batch_number :  10
batch_number :  11
batch_number :  12
batch_number :  13
batch_number :  14
batch_number :  15
batch_number :  16
batch_number :  17
batch_number :  18
batch_number :  19
batch_number :  20
batch_number :  21
batch_number :  22
batch_number :  23
batch_number :  24
batch_number :  25
batch_number :  26
batch_number :  27
batch_number :  28
batch_number :  29
batch_number :  30
batch_number :  31
batch_number :  32
batch_number :  33
batch_number :  34
batch_number :  35
batch_number :  36
batch_number :  37
batch_number :  38
batch_number :  39
batch_number :  40
batch_number :  41
batch_number :  42
batch_number :  43


  3%|▎         | 3/100 [00:32<17:30, 10.83s/it]


KeyboardInterrupt: 

In [10]:
import torch
from torch.utils.data import DataLoader
torch_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load("C:/Users/syed_fahim_ahmed/Desktop/Coding_With_Fahim/Unet_MC/TopoSegNetSimple/Output/Unet_Training_BS_10_ep_50/checkpoints/Unet_Training_BS_10_ep_50_49.pth", map_location=torch_device)

AttributeError: module 'torch' has no attribute '_utils'

In [7]:
import torch
torch.cuda.empty_cache()
torch.autograd.set_detect_anomaly(True)

In [8]:
import torch
torch.autograd.set_detect_anomaly(False)